# GPT 고객 서비스 챗봇 만들기

In [1]:
# 터미널에서 실행시
# python -m venv venv
# source venv/bin/activate  # 윈도우는 venv\Scripts\activate
# pip install -r requirements.txt
# python app.py

In [1]:
from dotenv import load_dotenv
import os

# .env 파일의 내용 불러오기
load_dotenv("C:/env/.env")

# 환경 변수 가져오기
API_KEY = os.getenv("OPENAI_API_KEY")

from openai import OpenAI
client = OpenAI(api_key=API_KEY)

from flask import Flask ,jsonify, request,render_template
app = Flask(__name__)

In [2]:
@app.route('/')
def index():
    return render_template('index.html')

@app.route('/get_response',methods=['POST'])
def get_response():
    user_message = request.json.get("message")

    # OpenAI API 호출
    response = client.chat.completions.create(
        model = "gpt-4o-mini",
        messages = [  {"role": "user",  "content": user_message} ],
        max_tokens = 300,
        n = 1,
        stop = None,
        temperature = 0.7,
    )

    bot_response = response.choices[0].message.content
    return jsonify({"response": bot_response})
                           
if __name__ == '__main__':
    app.run(debug=False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


### 질문한 내용을 Context로 계속 유지하면서 질문하기 

In [2]:
# 소스 코드를 이어서 실행할 경우 오류나므로 "Restart Kernel"을 수행하고 실행한다
@app.route('/')
def index():
    return render_template('index.html')

# 대화 히스토리를 저장할 변수
conversation_history = []

@app.route('/get_response',methods=['POST'])
def get_response():
    user_message = request.json.get("message")
    conversation_history.append({"role": "user",  "content": user_message})
    print(conversation_history)

    # OpenAI API 호출
    response = client.chat.completions.create(
        model = "gpt-4o-mini",
        messages = conversation_history,
        max_tokens = 300,
        n = 1,
        stop = None,
        temperature = 0.7,
    )

    bot_response = response.choices[0].message.content
    return jsonify({"response": bot_response})
                           
if __name__ == '__main__':
    app.run(debug=False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [26/Sep/2025 11:43:58] "GET / HTTP/1.1" 200 -


[{'role': 'user', 'content': '하늘이 파란 이유는?'}]


127.0.0.1 - - [26/Sep/2025 11:44:11] "POST /get_response HTTP/1.1" 200 -


[{'role': 'user', 'content': '하늘이 파란 이유는?'}, {'role': 'user', 'content': '노란 이유는?'}]


...

127.0.0.1 - - [26/Sep/2025 11:45:11] "POST /get_response HTTP/1.1" 200 -


[{'role': 'user', 'content': '하늘이 파란 이유는?'}, {'role': 'user', 'content': '노란 이유는?'}, {'role': 'user', 'content': '빨간이유는?'}, {'role': 'user', 'content': '나는 홍길동이야'}, {'role': 'user', 'content': '내 친구는 임꺽정이야'}]


127.0.0.1 - - [26/Sep/2025 11:45:49] "POST /get_response HTTP/1.1" 200 -


[{'role': 'user', 'content': '하늘이 파란 이유는?'}, {'role': 'user', 'content': '노란 이유는?'}, {'role': 'user', 'content': '빨간이유는?'}, {'role': 'user', 'content': '나는 홍길동이야'}, {'role': 'user', 'content': '내 친구는 임꺽정이야'}, {'role': 'user', 'content': '그러면 임꺽정의 친구의 이름은?'}]


127.0.0.1 - - [26/Sep/2025 11:46:20] "POST /get_response HTTP/1.1" 200 -


### 질문과 답변한 내용을 Context로 계속 유지하면서 질문하기 

In [ ]:
# 소스 코드를 이어서 실행할 경우 오류나므로 "Restart Kernel"을 수행하고 실행한다
@app.route('/')
def index():
    return render_template('index.html')

# 대화 히스토리를 저장할 변수
conversation_history = []

@app.route('/get_response',methods=['POST'])
def get_response():
    user_message = request.json.get("message")

    context = f"대화 내용을 참조해서 답변하세요 : {conversation_history}"
    
    # OpenAI API 호출
    response = client.chat.completions.create(
        model = "gpt-4o-mini",
        messages = [  {"role": "system", "content": context},
                      {"role": "user", "content": user_message} ], 
        max_tokens = 300,
        n = 1,
        stop = None,
        temperature = 0.7,
    )

    bot_response = response.choices[0].message.content

    # 대화 내용을 history에 저장
    conversation_history.append({"질문:": user_message,  "답변:":bot_response})
    print('history:\n', conversation_history,'\n')
    print('-'*30) 

    return jsonify({"response": bot_response})
                           
if __name__ == '__main__':
    app.run(debug=False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [26/Sep/2025 12:34:00] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/Sep/2025 12:34:09] "POST /get_response HTTP/1.1" 200 -


history:
 [{'질문:': '나는 홍길동이야', '답변:': '안녕하세요, 홍길동님! 어떻게 도와드릴까요?'}] 

------------------------------


127.0.0.1 - - [26/Sep/2025 12:34:19] "POST /get_response HTTP/1.1" 200 -


history:
 [{'질문:': '나는 홍길동이야', '답변:': '안녕하세요, 홍길동님! 어떻게 도와드릴까요?'}, {'질문:': '내이름이 뭐지?', '답변:': '안녕하세요! 당신의 이름은 홍길동입니다. 어떻게 도와드릴까요?'}] 

------------------------------


...

history:
 [{'질문:': '나는 홍길동이야', '답변:': '안녕하세요, 홍길동님! 어떻게 도와드릴까요?'}, {'질문:': '내이름이 뭐지?', '답변:': '안녕하세요! 당신의 이름은 홍길동입니다. 어떻게 도와드릴까요?'}, {'질문:': '하늘이 파란 이유는?', '답변:': '하늘이 파란 이유는 대기 중의 산란 현상 때문입니다. 태양빛은 여러 색깔의 빛이 혼합되어 있는데, 그 중 파란색 빛이 다른 색에 비해 파장이 짧아서 대기 중의 분자와 더 많이 산란됩니다. 이 때문에 우리는 하늘을 파랗게 보게 됩니다.'}, {'질문:': '노란 이유는?', '답변:': '하늘이 노란 이유는 주로 태양의 위치와 대기 중의 먼지, 수증기, 오염물질 등이 영향을 미칩니다. 태양이 낮은 각도에서 빛을 비추면, 빛이 대기를 통과하는 거리가 길어지면서 파장이 긴 색인 노란색과 빨간색 빛이 산란되어 하늘이 노랗게 보일 수 있습니다. 특히 일출이나 일몰 때 이런 현상이 두드러지게 나타납니다.'}, {'질문:': '빨간 이유는?', '답변:': '하늘이 빨간 이유는 주로 태양의 위치와 대기 중의 산란 현상 때문입니다. 태양이 지평선 가까이에 있을 때, 즉 일출이나 일몰 때, 태양빛이 대기를 지나가는 거리가 길어지면서 파장이 긴 색인 빨간색과 노란색 빛이 더 많이 산란됩니다. 이로 인해 하늘이 붉게 보이게 됩니다. 대기 중의 먼지나 오염물질도 이러한 색의 변화에 영향을 줄 수 있습니다.'}, {'질문:': '내 이름이 뭐지?', '답변:': '안녕하세요! 당신의 이름은 홍길동입니다. 어떻게 도와드릴까요?'}, {'질문:': '내 친구 이름은 임꺽정이야 ', '답변:': '안녕하세요! 친구분의 이름은 임꺽정이군요. 친구와 함께 즐거운 시간 보내세요! 어떻게 도와드릴까요?'}, {'질문:': '임꺽정의 친구의 이름은?', '답변:': '임꺽정의 친구의 이름은 명시되어 있지 않지만, 당신이 언급한 친구의 이름은 임꺽정입니다. 추가로 궁금한 점이 있으면 말씀해 주세요!'}, {'질

127.0.0.1 - - [26/Sep/2025 12:45:12] "POST /get_response HTTP/1.1" 200 -


history:
 [{'질문:': '나는 홍길동이야', '답변:': '안녕하세요, 홍길동님! 어떻게 도와드릴까요?'}, {'질문:': '내이름이 뭐지?', '답변:': '안녕하세요! 당신의 이름은 홍길동입니다. 어떻게 도와드릴까요?'}, {'질문:': '하늘이 파란 이유는?', '답변:': '하늘이 파란 이유는 대기 중의 산란 현상 때문입니다. 태양빛은 여러 색깔의 빛이 혼합되어 있는데, 그 중 파란색 빛이 다른 색에 비해 파장이 짧아서 대기 중의 분자와 더 많이 산란됩니다. 이 때문에 우리는 하늘을 파랗게 보게 됩니다.'}, {'질문:': '노란 이유는?', '답변:': '하늘이 노란 이유는 주로 태양의 위치와 대기 중의 먼지, 수증기, 오염물질 등이 영향을 미칩니다. 태양이 낮은 각도에서 빛을 비추면, 빛이 대기를 통과하는 거리가 길어지면서 파장이 긴 색인 노란색과 빨간색 빛이 산란되어 하늘이 노랗게 보일 수 있습니다. 특히 일출이나 일몰 때 이런 현상이 두드러지게 나타납니다.'}, {'질문:': '빨간 이유는?', '답변:': '하늘이 빨간 이유는 주로 태양의 위치와 대기 중의 산란 현상 때문입니다. 태양이 지평선 가까이에 있을 때, 즉 일출이나 일몰 때, 태양빛이 대기를 지나가는 거리가 길어지면서 파장이 긴 색인 빨간색과 노란색 빛이 더 많이 산란됩니다. 이로 인해 하늘이 붉게 보이게 됩니다. 대기 중의 먼지나 오염물질도 이러한 색의 변화에 영향을 줄 수 있습니다.'}, {'질문:': '내 이름이 뭐지?', '답변:': '안녕하세요! 당신의 이름은 홍길동입니다. 어떻게 도와드릴까요?'}, {'질문:': '내 친구 이름은 임꺽정이야 ', '답변:': '안녕하세요! 친구분의 이름은 임꺽정이군요. 친구와 함께 즐거운 시간 보내세요! 어떻게 도와드릴까요?'}, {'질문:': '임꺽정의 친구의 이름은?', '답변:': '임꺽정의 친구의 이름은 명시되어 있지 않지만, 당신이 언급한 친구의 이름은 임꺽정입니다. 추가로 궁금한 점이 있으면 말씀해 주세요!'}, {'질

127.0.0.1 - - [26/Sep/2025 12:45:40] "POST /get_response HTTP/1.1" 200 -


history:
 [{'질문:': '나는 홍길동이야', '답변:': '안녕하세요, 홍길동님! 어떻게 도와드릴까요?'}, {'질문:': '내이름이 뭐지?', '답변:': '안녕하세요! 당신의 이름은 홍길동입니다. 어떻게 도와드릴까요?'}, {'질문:': '하늘이 파란 이유는?', '답변:': '하늘이 파란 이유는 대기 중의 산란 현상 때문입니다. 태양빛은 여러 색깔의 빛이 혼합되어 있는데, 그 중 파란색 빛이 다른 색에 비해 파장이 짧아서 대기 중의 분자와 더 많이 산란됩니다. 이 때문에 우리는 하늘을 파랗게 보게 됩니다.'}, {'질문:': '노란 이유는?', '답변:': '하늘이 노란 이유는 주로 태양의 위치와 대기 중의 먼지, 수증기, 오염물질 등이 영향을 미칩니다. 태양이 낮은 각도에서 빛을 비추면, 빛이 대기를 통과하는 거리가 길어지면서 파장이 긴 색인 노란색과 빨간색 빛이 산란되어 하늘이 노랗게 보일 수 있습니다. 특히 일출이나 일몰 때 이런 현상이 두드러지게 나타납니다.'}, {'질문:': '빨간 이유는?', '답변:': '하늘이 빨간 이유는 주로 태양의 위치와 대기 중의 산란 현상 때문입니다. 태양이 지평선 가까이에 있을 때, 즉 일출이나 일몰 때, 태양빛이 대기를 지나가는 거리가 길어지면서 파장이 긴 색인 빨간색과 노란색 빛이 더 많이 산란됩니다. 이로 인해 하늘이 붉게 보이게 됩니다. 대기 중의 먼지나 오염물질도 이러한 색의 변화에 영향을 줄 수 있습니다.'}, {'질문:': '내 이름이 뭐지?', '답변:': '안녕하세요! 당신의 이름은 홍길동입니다. 어떻게 도와드릴까요?'}, {'질문:': '내 친구 이름은 임꺽정이야 ', '답변:': '안녕하세요! 친구분의 이름은 임꺽정이군요. 친구와 함께 즐거운 시간 보내세요! 어떻게 도와드릴까요?'}, {'질문:': '임꺽정의 친구의 이름은?', '답변:': '임꺽정의 친구의 이름은 명시되어 있지 않지만, 당신이 언급한 친구의 이름은 임꺽정입니다. 추가로 궁금한 점이 있으면 말씀해 주세요!'}, {'질